### Training Phase

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [2]:
attributes = pd.read_csv("attributes.csv", error_bad_lines=False, warn_bad_lines=False)

In [3]:
len(attributes)

184880449

In [4]:
attributes.attribute.unique()

array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 48, 52, 75, 76, 77, 78, 66, 96, 67, 49])

In [5]:
attributes.app_id.unique()

array([4724682771660800])

In [6]:
attributes = attributes.drop(['app_id'], axis=1)
attributes.head()

,session_id,attribute,attribute_value,user_id_hash
0,2201961907282901522,0,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,1,1,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,2201961907282901522,3,0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,0.00,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,5,romance_the_royal_romance_02:11,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [29]:
attributes['user_id_hash'].nunique()

622641

In [31]:
print(sessions['user_id_hash'].nunique())
print(events['user_id_hash'].nunique())

621106
621001


In [7]:
sessions = pd.read_csv("sessions.csv")
sessions = sessions.drop(['app_id'], axis=1)
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,is_session,is_developer,is_wau,...,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,True,False,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,1543712977293,Asia/Manila,28800000.0,35050130,1538874289458,False,True,False,False,...,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,True,False,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,True,False,False,...,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,True,False,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [8]:
sessions = sessions[(sessions.is_session == True) & (sessions.is_wau == False) & (sessions.is_mau == False) & (sessions.is_developer == False)]
sessions = sessions.drop(['is_session','is_developer','is_wau','is_mau'], axis=1)

In [9]:
sessions.head(3)

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,2201961907282901522,1543712977293,Asia/Manila,28800000.0,35050130,1538874289458,False,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [10]:
sessions.columns

Index(['session_id', 'start_timestamp', 'timezone', 'timezone_offset',
       'previous_sessions_duration', 'user_created_timestamp',
       'is_user_first_session', 'country', 'region', 'city', 'latitude',
       'longitude', 'locale', 'os_name', 'session_index', 'device_id',
       'user_id_hash'],
      dtype='object')

In [11]:
events = pd.read_csv("events.csv")
events = events.drop(['app_id'], axis=1)
events.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [12]:
a = attributes['user_id_hash'].unique()
b = sessions['user_id_hash'].unique()
c = events['user_id_hash'].unique()
end = np.concatenate([a,b,c])
uniq = set(end)

In [135]:
the_col = [i[1] for i in list(name2idx.items())]

In [13]:
name2idx = {o:i for i,o in enumerate(uniq)}
new_col = np.array([name2idx[x] for x in attributes['user_id_hash']])
attributes['user_id_hash'] = new_col

In [14]:
new_col = np.array([name2idx[x] for x in sessions['user_id_hash']])
sessions['user_id_hash'] = new_col

In [16]:
new_col = np.array([name2idx[x] for x in events['user_id_hash']])
events['user_id_hash'] = new_col

In [19]:
attributes_1314 = attributes[(attributes['attribute']==13)|(attributes['attribute']==14)]

In [20]:
attributes_1314.head()

,session_id,attribute,attribute_value,user_id_hash
11,2201961907282901522,13,3.49,514036
12,2201961907282901522,14,3.49,514036
37,2250719844051320866,13,0.00,514036
38,2250719844051320866,14,0.00,514036
63,5076942931271552500,13,3.49,514036


In [21]:
sess_att = pd.merge(sessions, attributes_1314,  how='outer', \
                          left_on=['session_id','user_id_hash'], right_on = ['session_id','user_id_hash'])
sess_att.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash,attribute,attribute_value
0,5558845121177764917,1.542215e+12,Asia/Manila,28800000.0,25837591.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,3.49
1,5558845121177764917,1.542215e+12,Asia/Manila,28800000.0,25837591.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,14,3.49
2,2201961907282901522,1.543713e+12,Asia/Manila,28800000.0,35050130.0,1.538874e+12,False,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,3.49
3,2201961907282901522,1.543713e+12,Asia/Manila,28800000.0,35050130.0,1.538874e+12,False,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,14,3.49
4,18781111175537580,1.539216e+12,Asia/Manila,28800000.0,11343848.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,0.00


In [47]:
oct_1 = datetime.datetime(2018, 10, 1, 23, 59, 59, 0)
oct_7 = datetime.datetime(2018, 10, 7, 23, 59, 59, 0)
oct_14 = datetime.datetime(2018, 10, 14, 23, 59, 59, 0)
oct_21 = datetime.datetime(2018, 10, 21, 23, 59, 59, 0)
oct_28 = datetime.datetime(2018, 10, 28, 23, 59, 59, 0)
nov_4 = datetime.datetime(2018, 11, 4, 23, 59, 59, 0)
nov_11 = datetime.datetime(2018, 11, 11, 23, 59, 59, 0)
nov_18 = datetime.datetime(2018, 11, 18, 23, 59, 59, 0)
nov_25 = datetime.datetime(2018, 11, 25, 23, 59, 59, 0)


nov_17 = datetime.datetime(2018, 11, 17, 23, 59, 59, 0)
nov_24 = datetime.datetime(2018, 11, 24, 23, 59, 59, 0)
dec_1 = datetime.datetime(2018, 12, 1, 23, 59, 59, 0)
dec_7 = datetime.datetime(2018, 12, 7, 23, 59, 59, 0)
dec_14 = datetime.datetime(2018, 12, 14, 23, 59, 59, 0)

In [23]:
# date time to milliseconds
def to_milli(t):
    return int(t.strftime("%s"))*1000

In [24]:
sess_att_train = sess_att[sess_att['start_timestamp']<to_milli(dec_1)]
sess_att_train.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash,attribute,attribute_value
0,5558845121177764917,1.542215e+12,Asia/Manila,28800000.0,25837591.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,3.49
1,5558845121177764917,1.542215e+12,Asia/Manila,28800000.0,25837591.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,14,3.49
2,2201961907282901522,1.543713e+12,Asia/Manila,28800000.0,35050130.0,1.538874e+12,False,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,3.49
3,2201961907282901522,1.543713e+12,Asia/Manila,28800000.0,35050130.0,1.538874e+12,False,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,14,3.49
4,18781111175537580,1.539216e+12,Asia/Manila,28800000.0,11343848.0,1.538874e+12,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10.0,546a3d98-d540-4e72-ad82-9ebd64e0839b,514036,13,0.00


In [26]:
sess_att_train['attribute_value'] = sess_att_train['attribute_value'].apply(lambda x: float(x))

/Users/ziyuanhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [108]:
outcome_13 = sess_att_train[sess_att_train['attribute']==13][['user_id_hash','start_timestamp','attribute_value']]\
                    .groupby(['user_id_hash'])\
                    .agg(lambda x: x.iloc[x['start_timestamp'].values.argmax()]).reset_index()
outcome_13.head()

,user_id_hash,start_timestamp,attribute_value
0,0,1.542998e+12,0.00
1,1,1.541243e+12,0.00
2,2,1.539347e+12,0.01
3,3,1.542305e+12,0.00
4,4,1.543405e+12,0.00


In [111]:
outcome_14 = sess_att_train[sess_att_train['attribute']==14]\
                    .groupby(['user_id_hash'])\
                    .agg(lambda x: x.iloc[x['start_timestamp'].values.argmax()]).reset_index()
outcome_14.head()

,user_id_hash,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,attribute,attribute_value,attribute_value_f
0,0,4325654999572362475,1.542998e+12,Europe/London,0.0,550360.0,1.542801e+12,False,GB,eng,bristol,51.454514,-2.587910,en_GB,Android OS,2.0,d46750fa-995e-4dde-81cf-9785af69230b,14,0.0,0.0
1,1,538779361517557750,1.541243e+12,Asia/Kolkata,19800000.0,0.0,1.541243e+12,True,IN,dl,delhi,28.686274,77.221786,en_IN,Android OS,1.0,06ad20ca-c866-4242-bca5-e71ad039e636,14,0.0,0.0
2,2,3618021605312627847,1.539347e+12,Europe/London,3600000.0,6974272.0,1.539272e+12,False,GB,sct,kilwinning,55.655918,-4.703117,en-GB_GB,iOS,6.0,205317ED-E358-402F-9568-C9AADE332699,14,0.0,0.0
3,3,5908698171697520519,1.542305e+12,America/Chicago,-21600000.0,0.0,1.542305e+12,True,US,tn,union city,36.424229,-89.057007,en_US,Android OS,1.0,a2d586ee-e484-481c-99bd-74156873a09d,14,0.0,0.0
4,4,756913619296302504,1.543405e+12,Europe/Stockholm,3600000.0,14777.0,1.543324e+12,False,SE,ab,stockholm,59.329323,18.068581,sv-SE_SE,iOS,2.0,952C8FDA-F729-4264-A1D8-8878A0473F46,14,0.0,0.0


In [110]:
sess_att_train['attribute_value'][0]

3.49

In [199]:
sess_att['attribute'].unique()

array([13, 14])

In [27]:
outcome_13_mean = sess_att_train[sess_att_train['attribute']==13]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('mean').reset_index()

outcome_14_mean = sess_att_train[sess_att_train['attribute']==14]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('mean').reset_index()

In [28]:
outcome_13_max = sess_att_train[sess_att_train['attribute']==13]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('max').reset_index()

outcome_14_max = sess_att_train[sess_att_train['attribute']==14]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('max').reset_index()

In [32]:
outcome_14_max.head(5)

,user_id_hash,attribute_value
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [38]:
bought.head(3)

,session_id,event,event_timestamp,event_value,user_id_hash
0,5159335150551901701,8,1541912600211,3.493,514036
1,845986603555615931,8,1543357433771,3.493,575129
2,2846996524173831068,8,1543023539172,3.493,575129


In [40]:
bought = pd.merge(bought, sessions[['user_id_hash','user_created_timestamp']], \
                          how='left', left_on='user_id_hash',right_on='user_id_hash')

In [41]:
bought.head(3)

,session_id,event,event_timestamp,event_value,user_id_hash,user_created_timestamp
0,5159335150551901701,8,1541912600211,3.493,514036,1.538874e+12
1,5159335150551901701,8,1541912600211,3.493,514036,1.538874e+12
2,5159335150551901701,8,1541912600211,3.493,514036,1.538874e+12


In [46]:
bought_1111_1118.head(3)

,user_id_hash,count
0,7,0
1,30,0
2,31,0


In [48]:
bought_1111_1118 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_11))&(x < to_milli(nov_18))).sum()).reset_index(name='count')
print(sum(bought_1111_1118['count']))


bought_1118_1125 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_18))&(x < to_milli(nov_25))).sum()).reset_index(name='count')
print(sum(bought_1118_1125['count']))


bought_1125_1201 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_25))&(x < to_milli(dec_1))).sum()).reset_index(name='count')
print(sum(bought_1125_1201['count']))

bought_sum = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(oct_1))&(x < to_milli(dec_1))).sum()).reset_index(name='count')
print(sum(bought_sum['count']))

1944394
1934062
1520236
14797251


In [51]:
# events_1111_1118 = events.groupby('user_id_hash')['event_timestamp']\
#                         .apply(lambda x: ((x > to_milli(nov_11))&(x < to_milli(nov_18))).sum()).reset_index(name='count')
# print(sum(events_1111_1118['count']))


# events_1118_1125 = events.groupby('user_id_hash')['event_timestamp']\
#                         .apply(lambda x: ((x > to_milli(nov_18))&(x < to_milli(nov_25))).sum()).reset_index(name='count')
# print(sum(events_1118_1125['count']))


events_1125_1201 = events.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_25))&(x < to_milli(dec_1))).sum()).reset_index(name='count')
print(sum(events_1125_1201['count']))

# events_sum = events.groupby('user_id_hash')['event_timestamp']\
#                         .apply(lambda x: ((x > to_milli(oct_1))&(x < to_milli(dec_1))).sum()).reset_index(name='count')
# print(sum(events_sum['count']))


9891204


In [58]:
events_1125_1201.head(3)

,user_id_hash,count
0,0,0
1,1,0
2,2,0


In [52]:
sessions_before_12_1 = sessions[sessions['start_timestamp']<to_milli(dec_1)]
user_num_ses_cre_and_tot_ses_time = sessions_before_12_1.groupby('user_id_hash')['previous_sessions_duration']\
                                                            .agg(['sum','count']).reset_index()

user_num_ses_cre_and_tot_ses_time['sum'] = user_num_ses_cre_and_tot_ses_time['sum']/60

user_num_ses_cre_and_tot_ses_time.head(5)
# .size().reset_index(name='counts')

,user_id_hash,sum,count
0,0,0.0,1
1,1,0.0,1
2,2,0.0,1
3,3,0.0,1
4,4,0.0,1


In [53]:
user_num_day_cre = sessions_before_12_1[['user_id_hash','user_created_timestamp']]\
                    .drop_duplicates(subset='user_id_hash', keep='first')\
                    .reset_index(drop=True)
print(len(user_num_day_cre))
print(len(user_num_day_cre['user_id_hash'].unique()))

619037
619037


In [54]:
user_num_day_cre['num_day_created'] = ((to_milli(dec_1)-user_num_day_cre['user_created_timestamp'])/(1000*60*60*24)) % 7
user_num_day_cre.head(3)

,user_id_hash,user_created_timestamp,num_day_created
0,514036,1538874289458,0.288305
1,317277,1541886003916,0.430499
2,11171,1539830331932,3.222998


In [55]:
user_ave = user_num_ses_cre_and_tot_ses_time.merge(user_num_day_cre, left_on='user_id_hash', right_on='user_id_hash')
# df = df.rename(index=str, columns={"count": "events_1125_1201"})
user_ave = user_ave.drop(['user_created_timestamp'],axis=1)
user_ave.head(3)

,user_id_hash,sum,count,num_day_created
0,0,0.0,1,0.793792
1,1,0.0,1,2.642547
2,2,0.0,1,2.717589


In [56]:
user_ave['average_minutes'] = user_ave['sum']/user_ave['num_day_created']
user_ave['average_counts'] = user_ave['count']/user_ave['num_day_created']
user_ave.head(3)

,user_id_hash,sum,count,num_day_created,average_minutes,average_counts
0,0,0.0,1,0.793792,0.0,1.259776
1,1,0.0,1,2.642547,0.0,0.378423
2,2,0.0,1,2.717589,0.0,0.367973


In [57]:
user_ave = user_ave.drop(['sum','count'], axis=1)
user_ave.head(3)

,user_id_hash,num_day_created,average_minutes,average_counts
0,0,0.793792,0.0,1.259776
1,1,2.642547,0.0,0.378423
2,2,2.717589,0.0,0.367973


In [69]:
# test_1 = pd.merge(df, outcome_13[['user_id_hash','attribute_value']], \
#                       how='left', left_on='user_id_hash',right_on='user_id_hash')
# test_1 = test_1.rename(index=str, columns={"attribute_value": "attribute_value_13"})

# test_1 = pd.merge(test_1, outcome_14[['user_id_hash','attribute_value']], \
#                       how='left', left_on='user_id_hash',right_on='user_id_hash')
# test_1 = test_1.rename(index=str, columns={"attribute_value": "attribute_value_14"})

# test_1 = test_1.fillna(0)
# test_1.head()

test_1 = pd.merge(df, outcome_13_max[['user_id_hash','attribute_value']], \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"attribute_value": "attribute_value_13_max"})

test_1 = pd.merge(test_1, outcome_14_max[['user_id_hash','attribute_value']], \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"attribute_value": "attribute_value_14_max"})

test_1 = pd.merge(test_1, bought_1111_1118, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"count": "count_1111_1118"})

test_1 = pd.merge(test_1, bought_1118_1125, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"count": "count_1118_1125"})

test_1 = pd.merge(test_1, bought_1125_1201, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"count": "count_1125_1201"})

test_1 = pd.merge(test_1, events_1125_1201, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
test_1 = test_1.rename(index=str, columns={"count": "event_1125_1201"})

test_1 = pd.merge(test_1, user_ave, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')


test_1 = test_1.fillna(0)
test_1.head()

,user_id_hash,dec1_14,dec1_7,attribute_value_13_max,attribute_value_14_max,count_1111_1118,count_1118_1125,count_1125_1201,event_1125_1201,num_day_created,average_minutes,average_counts
0,514036,0,0,3.49,3.49,0.0,0.0,0.0,51,0.288305,5.838944e+07,166.490416
1,317277,0,0,0.01,0.00,0.0,0.0,0.0,0,0.430499,2.351852e+05,6.968663
2,11171,0,0,0.00,0.00,0.0,0.0,0.0,0,3.222998,0.000000e+00,0.310270
3,90676,0,0,0.00,0.00,0.0,0.0,0.0,0,0.057656,0.000000e+00,17.344389
4,76763,0,0,0.00,0.00,0.0,0.0,0.0,0,5.672814,0.000000e+00,0.176279


In [72]:
test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1).shape

(621001, 9)

In [222]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_7']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf_dec1_7 = GradientBoostingClassifier(loss = 'exponential', min_samples_split = 50, \
                                        min_samples_leaf=10)
clf_dec1_7.fit(X_train, y_train)
y_scores = clf_dec1_7.predict_proba(X_test)
roc_auc_score(y_test, y_scores[:,1])

0.9722152189319492

In [221]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_14']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf_dec1_14 = GradientBoostingClassifier(loss = 'exponential', min_samples_split = 50, \
                                         min_samples_leaf=10)
clf_dec1_14.fit(X_train, y_train)
y_scores = clf_dec1_14.predict_proba(X_test)
roc_auc_score(y_test, y_scores[:,1])

0.9625064162207312

In [187]:
# from sklearn.model_selection import GridSearchCV

# param_grid = dict(loss = ['deviance', 'exponential'],
#                   learning_rate=[0.1, 0.4],
#                   n_estimators = [100],
#                   subsample = [0.5, 0.75, 1],
#                   min_samples_split = [2,5,50],
#                   max_depth = [3,5],
#                   min_samples_leaf=[1,3,5])

# gbc_cv = GridSearchCV(estimator=GradientBoostingClassifier(), 
#                       param_grid=param_grid, 
#                       cv=5)
# gbc_cv.fit(X, y)
# print(gbc_cv.best_params_)
# print(f"{gbc_cv.best_score_:,.4f}")

#### training models

In [223]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_7']
clf_dec1_7 = GradientBoostingClassifier(loss = 'exponential', min_samples_split = 50, \
                                        min_samples_leaf=10)
clf_dec1_7.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=10, min_samples_split=50,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [108]:
from sklearn.ensemble import RandomForestClassifier
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_7']
clf_dec1_7 = RandomForestClassifier(n_estimators=100)
clf_dec1_7.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [126]:
import xgboost as xgb
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_7']
clf_dec1_7 = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
clf_dec1_7.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [224]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_14']
clf_dec1_14 = GradientBoostingClassifier(loss = 'exponential', min_samples_split = 50, \
                                        min_samples_leaf=10)
clf_dec1_14.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=10, min_samples_split=50,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [109]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_14']
clf_dec1_14 = RandomForestClassifier(n_estimators=100)
clf_dec1_14.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [127]:
X = test_1.drop(['user_id_hash','dec1_7','dec1_14'], axis=1)
y = test_1['dec1_14']
clf_dec1_14 = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
clf_dec1_14.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [81]:
import pickle
# save the classifier
with open('clf_dec1_7.pkl', 'wb') as fid:
    pickle.dump(clf_dec1_7, fid)    

with open('clf_dec1_14.pkl', 'wb') as fid:
    pickle.dump(clf_dec1_14, fid)  

    
# load it again
# with open('my_dumped_classifier.pkl', 'rb') as fid:
#     gnb_loaded = cPickle.load(fid)

#### Create id-labels

In [33]:
bought = events[events['event']=='8'].reset_index()
bought = bought.drop(['index'],axis=1)

In [34]:
# bought after 12.1
# already_bought = []
bought_after_12_1 = bought[bought['event_timestamp']>to_milli(dec_1)]['user_id_hash'].unique()
print(bought_after_12_1[:3])
# bought after 12.1 before 12.7
bought_between_12_1_7 = bought[(bought['event_timestamp']>to_milli(dec_1)) & (bought['event_timestamp']<to_milli(dec_7))]['user_id_hash'].unique()
print(bought_between_12_1_7[:3])

[545157 235308 431370]
[545157 235308 431370]


In [60]:
df = events['user_id_hash'].unique()
df = pd.DataFrame(data = df, columns=['user_id_hash'])

In [61]:
df['dec1_14'] = df['user_id_hash'].apply(lambda x: 1 if x in bought_after_12_1 else 0)
df['dec1_7'] = df['user_id_hash'].apply(lambda x: 1 if x in bought_between_12_1_7 else 0)

In [62]:
df.head(3)

,user_id_hash,dec1_14,dec1_7
0,514036,0,0
1,317277,0,0
2,11171,0,0


### Feature engineering for predicting phase

In [73]:
sess_att['attribute_value'] = sess_att['attribute_value'].apply(lambda x: float(x))

In [82]:
outcome_13_pred = sess_att[sess_att['attribute']==13]\
                    .groupby(['user_id_hash'])\
                    .agg(lambda x: x.iloc[x['start_timestamp'].values.argmax()]).reset_index()
outcome_13_pred.head()

,user_id_hash,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,attribute,attribute_value
0,0,4392535094850097365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,0.0
1,1,266660243141322578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,0.0
2,2,7786193078488883616,1.541084e+12,Asia/Kuala_Lumpur,28800000.0,0.0,1.541084e+12,True,MY,14,kuala lumpur,3.139003,101.686852,en_GB,Android OS,1.0,094aa744-eea2-4db0-a1c7-7bbf08d3e2de,13,0.0
3,3,2664398413429705038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,0.0
4,4,7238754199158869779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,0.0


In [83]:
outcome_14_pred = sess_att[sess_att['attribute']==14]\
                    .groupby(['user_id_hash'])\
                    .agg(lambda x: x.iloc[x['start_timestamp'].values.argmax()]).reset_index()
outcome_14_pred.head()

,user_id_hash,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,attribute,attribute_value
0,0,4392535094850097365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,0.0
1,1,266660243141322578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,0.0
2,2,7786193078488883616,1.541084e+12,Asia/Kuala_Lumpur,28800000.0,0.0,1.541084e+12,True,MY,14,kuala lumpur,3.139003,101.686852,en_GB,Android OS,1.0,094aa744-eea2-4db0-a1c7-7bbf08d3e2de,14,0.0
3,3,2664398413429705038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,0.0
4,4,7238754199158869779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,0.0


In [135]:
outcome_13_pred_mean = sess_att[sess_att['attribute']==13]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('mean').reset_index()

outcome_14_pred_mean = sess_att[sess_att['attribute']==14]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('mean').reset_index()

In [84]:
outcome_13_pred_max = sess_att[sess_att['attribute']==13]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('max').reset_index()

outcome_14_pred_max = sess_att[sess_att['attribute']==14]\
                    .groupby(['user_id_hash'])['attribute_value']\
                    .agg('max').reset_index()

In [225]:
# col = sess_att['user_id_hash'].unique()
col = the_col
final_df = pd.DataFrame(data = col, columns=['user_id_hash'])

In [226]:
len(final_df)

622681

In [146]:
nov_23 = datetime.datetime(2018, 11, 23, 23, 59, 59, 0)
nov_30 = datetime.datetime(2018, 11, 30, 23, 59, 59, 0)
dec_7 = datetime.datetime(2018, 12, 7, 23, 59, 59, 0)

bought_1123_1130 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_23))&(x < to_milli(nov_30))).sum()).reset_index(name='count')
print(sum(bought_1123_1130['count']))

bought_1130_1207 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(nov_30))&(x < to_milli(dec_7))).sum()).reset_index(name='count')
print(sum(bought_1130_1207['count']))


bought_1207_1214 = bought.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(dec_7))&(x < to_milli(dec_14))).sum()).reset_index(name='count')
print(sum(bought_1207_1214['count']))

1856256
1421153
1015267


In [147]:
events_1207_1214 = events.groupby('user_id_hash')['event_timestamp']\
                        .apply(lambda x: ((x > to_milli(dec_7))&(x < to_milli(dec_14))).sum()).reset_index(name='count')
print(sum(events_1207_1214['count']))

6157769


In [148]:
session_after_10_14 = sessions[sessions['start_timestamp']>to_milli(oct_14)]

In [149]:
user_num_ses_cre_and_tot_ses_time = session_after_10_14.groupby('user_id_hash')['previous_sessions_duration']\
                                                            .agg(['sum','count']).reset_index()

user_num_ses_cre_and_tot_ses_time['sum'] = user_num_ses_cre_and_tot_ses_time['sum']/60

user_num_ses_cre_and_tot_ses_time.head(5)

,user_id_hash,sum,count
0,0,0.0,1
1,1,0.0,1
2,2,0.0,1
3,3,0.0,1
4,4,0.0,1


In [150]:
user_num_day_cre = session_after_10_14[['user_id_hash','user_created_timestamp']]\
                    .drop_duplicates(subset='user_id_hash', keep='first')\
                    .reset_index(drop=True)
print(len(user_num_day_cre))
print(len(user_num_day_cre['user_id_hash'].unique()))

517219
517219


In [151]:
user_num_day_cre['num_day_created'] = ((to_milli(dec_14)-user_num_day_cre['user_created_timestamp'])/(1000*60*60*24)) % 7
user_num_day_cre.head(3)

,user_id_hash,user_created_timestamp,num_day_created
0,514036,1538874289458,6.288305
1,317277,1541886003916,6.430499
2,11171,1539830331932,2.222998


In [152]:
user_ave = user_num_ses_cre_and_tot_ses_time.merge(user_num_day_cre, left_on='user_id_hash', right_on='user_id_hash')
user_ave = user_ave.drop(['user_created_timestamp'],axis=1)
user_ave.head(3)

,user_id_hash,sum,count,num_day_created
0,0,0.0,1,6.793792
1,1,0.0,1,1.642547
2,2,0.0,1,1.717589


In [153]:
user_ave['average_minutes'] = user_ave['sum']/user_ave['num_day_created']
user_ave['average_counts'] = user_ave['count']/user_ave['num_day_created']
user_ave.head(3)

,user_id_hash,sum,count,num_day_created,average_minutes,average_counts
0,0,0.0,1,6.793792,0.0,0.147193
1,1,0.0,1,1.642547,0.0,0.608810
2,2,0.0,1,1.717589,0.0,0.582211


In [154]:
user_ave = user_ave.drop(['sum','count'], axis=1)
user_ave.head(3)

,user_id_hash,num_day_created,average_minutes,average_counts
0,0,6.793792,0.0,0.147193
1,1,1.642547,0.0,0.608810
2,2,1.717589,0.0,0.582211


In [227]:
# final_df = pd.merge(final_df, outcome_13_pred[['user_id_hash','attribute_value']], \
#                       how='left', left_on='user_id_hash',right_on='user_id_hash')
# final_df = final_df.rename(index=str, columns={"attribute_value": "attribute_value_13"})

# final_df = pd.merge(final_df, outcome_14_pred[['user_id_hash','attribute_value']], \
#                       how='left', left_on='user_id_hash',right_on='user_id_hash')
# final_df = final_df.rename(index=str, columns={"attribute_value": "attribute_value_14"})

final_df = pd.merge(final_df, outcome_13_pred_max[['user_id_hash','attribute_value']], \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"attribute_value": "attribute_value_13_max"})

final_df = pd.merge(final_df, outcome_14_pred_max[['user_id_hash','attribute_value']], \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"attribute_value": "attribute_value_14_max"})

final_df = pd.merge(final_df, bought_1123_1130, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"count": "count_1111_1118"})

final_df = pd.merge(final_df, bought_1130_1207, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"count": "count_1118_1125"})

final_df = pd.merge(final_df, bought_1207_1214, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"count": "count_1125_1201"})

final_df = pd.merge(final_df, events_1207_1214, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')
final_df = final_df.rename(index=str, columns={"count": "event_1125_1201"})

final_df = pd.merge(final_df, user_ave, \
                      how='left', left_on='user_id_hash',right_on='user_id_hash')

final_df = final_df.fillna(0)
final_df.head()

,user_id_hash,attribute_value_13_max,attribute_value_14_max,count_1111_1118,count_1118_1125,count_1125_1201,event_1125_1201,num_day_created,average_minutes,average_counts
0,0,0.0,0.0,0.0,0.0,0.0,0.0,6.793792,0.0,0.147193
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.642547,0.0,0.608810
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.717589,0.0,0.582211
3,3,0.0,0.0,0.0,0.0,0.0,0.0,2.680869,0.0,0.373013
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.207004,0.0,4.830819


In [175]:
# final_df['attribute_value_13'] = final_df['attribute_value_13'].apply(lambda x: float(x))
# final_df['attribute_value_14'] = final_df['attribute_value_14'].apply(lambda x: float(x))

In [228]:
final_df['pred_7'] = clf_dec1_7.predict_proba(final_df.iloc[:,1:])[:,1]
final_df['pred_14'] = clf_dec1_14.predict_proba(final_df.iloc[:,1:-1])[:,1]

In [229]:
final_df.head()

,user_id_hash,attribute_value_13_max,attribute_value_14_max,count_1111_1118,count_1118_1125,count_1125_1201,event_1125_1201,num_day_created,average_minutes,average_counts,pred_7,pred_14
0,0,0.0,0.0,0.0,0.0,0.0,0.0,6.793792,0.0,0.147193,0.000084,0.000206
1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.642547,0.0,0.608810,0.000126,0.000280
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.717589,0.0,0.582211,0.000126,0.000280
3,3,0.0,0.0,0.0,0.0,0.0,0.0,2.680869,0.0,0.373013,0.000134,0.000280
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.207004,0.0,4.830819,0.000115,0.000289


In [230]:
answer = final_df[['user_id_hash','pred_7','pred_14']]

### Prediction

In [231]:
submission = pd.read_csv('sample_submission_2.csv')
submission.head(3)

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02


In [232]:
def tryconvert(value):
    try:
        return name2idx[value]
    except KeyError:
        return value

In [233]:
submission['user'] = submission['user_id_hash'].apply(lambda x: tryconvert(x))
submission.head(3)

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days,user
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02,332490
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02,590690
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02,163489


In [234]:
answer = answer.rename(index=str, columns={"user_id_hash": "user"})
answer.head(3)

,user,pred_7,pred_14
0,0,0.000084,0.000206
1,1,0.000126,0.000280
2,2,0.000126,0.000280


In [235]:
final = submission.merge(answer, on='user', how='left').fillna(0)
final.head(3)

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days,user,pred_7,pred_14
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02,332490,0.000428,0.000952
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02,590690,0.000715,0.001404
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02,163489,0.000185,0.000389


In [236]:
final = final.drop(['user','user_purchase_binary_7_days','user_purchase_binary_14_days'],axis=1)
final = final.rename(index=str, columns={"pred_7": "user_purchase_binary_7_days", "pred_14":"user_purchase_binary_14_days"})
final.to_csv("final2.csv",index=False)

In [237]:
final = pd.read_csv('final2.csv')
final.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.000428,0.000952
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.000715,0.001404
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.000185,0.000389
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.001032,0.001839
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.000973,0.003514
